In [1]:
import pickle
import os
import pandas as pd
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
print(BASE_DIR)

import scripts.OrderBookAnalizer as oba
import scripts.functions as fn

c:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app\backtest
C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


## Revision del ultimo conjunto de datos obtenido

In [ ]:
symbol='BTCUSDT'
LOG_DIR = os.path.join(BASE_DIR,'log')
DATA_FILE = os.path.join(LOG_DIR, f"order_book_{symbol}.pkl")
if os.path.exists(DATA_FILE):
    with open(DATA_FILE, "rb") as archivo:
        depth = pickle.load(archivo)

analizer = oba.OrderBookAnalyzer()
record = analizer.analyze_order_book(depth['bids'], depth['asks'])
for r in record['bid_supports']['levels']:
    print(r['price_group'],' - ',round(r['volume_pct'],1))

#print(record['price_scale_used'])

KeyError: 'bid_resistances'

## Revision del historico

In [3]:
symbol='BTCUSDT'
LOG_DIR = os.path.join(BASE_DIR,'log')
DATA_FILE = os.path.join(LOG_DIR, f"order_book_{symbol}_historic.pkl")
if os.path.exists(DATA_FILE):
    with open(DATA_FILE, "rb") as archivo:
        df = pickle.load(archivo)

## Analisis de los resultados

In [4]:
qty_recs = df['timestamp'].count()
if qty_recs>48:
    qty_recs = 48
print(f'Total de registros almacenados: {qty_recs}')
print('Tiempo de datos almacenados: ',df.iloc[-1]['timestamp']-df.iloc[0]['timestamp'])

#Primer analisis sobre ultima lectura de ordenes
analysis_result = df.iloc[-1]
imbalance = analysis_result['market_imbalance']['imbalance_pct']

if imbalance > 25:
    print("Fuerte presión COMPRADORA - Posible movimiento alcista")
elif imbalance > 10:
    print("Moderada presión compradora")
elif imbalance < -25:
    print("Fuerte presión VENDEDORA - Posible movimiento bajista")
elif imbalance < -10:
    print("Moderada presión vendedora")
else:
    print("Mercado equilibrado")

analizer = oba.OrderBookAnalyzer(DATA_FILE)
summary = analizer.get_summary_stats(qty_recs)

print("Soportes clave históricos:")
for level in summary['support_levels']:
    print(f"• ${level['mean_price']:.2f}: "
          f"{level['mean_volume_pct']:.2f}% volumen "
          f"(apareció {level['frequency']} veces)")
    
print("Resistencias clave históricos:")
for level in summary['resistance_levels']:
    print(f"• ${level['mean_price']:.2f}: "
          f"{level['mean_volume_pct']:.2f}% volumen "
          f"(apareció {level['frequency']} veces)")
    
change = summary['price_change_pct']
direction = "subió" if change > 0 else "bajó"
print(f"El precio {direction} un {abs(change):.2f}% en los últimas {qty_recs} periodos")


Total de registros almacenados: 48
Tiempo de datos almacenados:  4 days 18:34:18.299845
Moderada presión compradora
Soportes clave históricos:
• $75000.00: 13.20% volumen (apareció 34 veces)
• $80296.30: 11.61% volumen (apareció 54 veces)
• $78000.00: 10.00% volumen (apareció 25 veces)
• $70000.00: 9.97% volumen (apareció 10 veces)
• $58944.00: 8.89% volumen (apareció 2 veces)
• $76000.00: 8.82% volumen (apareció 11 veces)
• $77000.00: 8.30% volumen (apareció 5 veces)
• $79114.29: 8.01% volumen (apareció 7 veces)
• $73972.00: 7.93% volumen (apareció 4 veces)
• $60000.00: 7.26% volumen (apareció 2 veces)
Resistencias clave históricos:
• $87582.69: 23.16% volumen (apareció 52 veces)
• $120000.00: 21.73% volumen (apareció 5 veces)
• $150000.00: 19.40% volumen (apareció 2 veces)
• $200000.00: 14.83% volumen (apareció 2 veces)
• $109600.00: 14.40% volumen (apareció 5 veces)
• $97213.61: 14.39% volumen (apareció 44 veces)
• $105000.00: 11.79% volumen (apareció 9 veces)
• $89809.98: 11.43% vo

In [5]:
analizer.analyze_summary(summary,qty_recs)


=== Análisis de los últimos 48 horas ===

Tendencia del mercado: Presión COMPRADORA (16.39% de desbalance)

Soportes más consistentes:
- $80296: 54 apariciones (Vol: 11.6%)
- $75000: 34 apariciones (Vol: 13.2%)
- $78000: 25 apariciones (Vol: 10.0%)

Resistencias más consistentes:
- $87583: 52 apariciones (Vol: 23.2%)
- $89810: 45 apariciones (Vol: 11.4%)
- $97214: 44 apariciones (Vol: 14.4%)

Divergencia detectada entre precio y volumen


In [6]:
print(f"\n📈 Resumen {qty_recs/2}h | Δ {summary['price_change_pct']:+.2f}%")
print(f"⚖️ Desbalance: {summary['mean_imbalance']:+.1f}%")
print(f"🛡️ Soportes clave: {len(summary['support_levels'])}")
print(f"🚧 Resistencias: {len(summary['resistance_levels'])}")

print("\n🔝 Top resistencias:")
for s in sorted(summary['resistance_levels'], key=lambda x: -x['mean_volume_pct'])[:3]:
    print(f"• ${s['mean_price']:.0f} | 📊 {s['mean_volume_pct']:.1f}% | 🔁 {s['frequency']}x")

print("\n🔝 Top soportes:")
for s in sorted(summary['support_levels'], key=lambda x: -x['mean_volume_pct'])[:3]:
    print(f"• ${s['mean_price']:.0f} | 📊 {s['mean_volume_pct']:.1f}% | 🔁 {s['frequency']}x")


📈 Resumen 24.0h | Δ -3.67%
⚖️ Desbalance: +16.4%
🛡️ Soportes clave: 10
🚧 Resistencias: 9

🔝 Top resistencias:
• $87583 | 📊 23.2% | 🔁 52x
• $120000 | 📊 21.7% | 🔁 5x
• $150000 | 📊 19.4% | 🔁 2x

🔝 Top soportes:
• $75000 | 📊 13.2% | 🔁 34x
• $80296 | 📊 11.6% | 🔁 54x
• $78000 | 📊 10.0% | 🔁 25x


## Estrategia

In [7]:
current_price = df['base_price'].iloc[-1]
nearest_support = min(summary['support_levels'], key=lambda x: abs(x['mean_price']-current_price))

if current_price < nearest_support['mean_price'] * 1.01:  # 1% sobre el soporte
    print("Posible rebote técnico")

Posible rebote técnico


## Generando series de S&R y Market Imbalance

In [8]:
sr = []
mi = []
miu = []
mid = []
for i,row in df.iterrows():
    tot_vol = row['total_bid_volume']+row['total_ask_volume']
    for r in row['bid_supports']['levels']:
        if r['price']>row['base_price']*0.8:
            sr.append({'timestamp': row['timestamp'],
                    'price': r['price'],
                    'pct': round((r['volume']/tot_vol)*100,0),
                    })
    for r in row['ask_resistances']['levels']:
        if r['price']<row['base_price']*1.2:
            sr.append({'timestamp': row['timestamp'],
                    'price': r['price'],
                    'pct': round((r['volume']/tot_vol)*100,0),
                    })
    mi.append({'timestamp':row['timestamp'],
               'mi':-round(row['market_imbalance']['imbalance_pct'],2)})
    if row['market_imbalance']['imbalance_pct']>0:
        miu.append({'timestamp':row['timestamp'],
                    'mi':-round(row['market_imbalance']['imbalance_pct'],2)})
    if row['market_imbalance']['imbalance_pct']<0:
        mid.append({'timestamp':row['timestamp'],
                    'mi':-round(row['market_imbalance']['imbalance_pct'],2)})
    
sr = pd.DataFrame(sr)
sr_pct_max = sr['pct'].max()
sr_pct_min = sr['pct'].min()
sr['pct_adj'] = fn.map(sr['pct'],sr_pct_min,sr_pct_max,1,12)

mi = pd.DataFrame(mi, columns=['timestamp','mi'])
mi['ma'] = mi['mi'].rolling(window=48).mean()
miu = pd.DataFrame(miu, columns=['timestamp','mi'])
mid = pd.DataFrame(mid, columns=['timestamp','mi'])

## Creando Chart

In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

chart_rows = 2
row_heights=[600,200]
total_height = sum(row_heights)

fig = make_subplots(rows=chart_rows, 
                    shared_xaxes=True,
                    row_heights=row_heights,
                    )

fig.add_trace(
        go.Scatter(
            x=df["timestamp"], y=df["base_price"], name=f'{symbol} Price', mode="lines",  
            line={'width': 0.75},  
            marker=dict(color='yellow',),
            legendgroup = '1',
            hovertemplate="%{x}<br>%{y}<extra></extra>",
        ),
        row=1,
        col=1,
    )  


fig.add_trace(
        go.Scatter(x=sr["timestamp"], y=sr['price'], name='Sop y Res', mode='markers', 
                marker=dict(symbol='circle',
                            size=sr['pct_adj'],
                            color=sr['pct_adj'],
                            colorscale="ice",
                            #showscale=True,
                            line=dict(width=0,),
                            ),
                legendgroup = '1',
                hovertemplate="%{x}<br>%{y}<extra></extra>",
                ),row=1,col=1,
)

fig.add_trace(
        go.Bar(
            x=miu["timestamp"], y=miu["mi"], name=f'{symbol}',  
            marker=dict(color='#f6465d',
                        line=dict(width=0,)
                        ),
            hovertemplate="%{x}<br>%{y} %<extra></extra>",
            legendgroup = '2',
            showlegend=False,
        ),
        row=2,
        col=1,
    )  
fig.add_trace(
        go.Bar(
            x=mid["timestamp"], y=mid["mi"], name=f'{symbol}',  
            marker=dict(color='#0ecb81',
                        line=dict(width=0,)
                        ),
            hovertemplate="%{x}<br>%{y} %<extra></extra>",
            legendgroup = '2',
            showlegend=False,
        ),
        row=2,
        col=1,
    ) 

fig.add_trace(
        go.Scatter(
            x=mi["timestamp"], y=mi["ma"], name=f'MI MA', mode="lines",  
            line={'width': 0.75},  
            marker=dict(color='white',),
            legendgroup = '1',
            hovertemplate="%{x}<br>%{y}<extra></extra>",
        ),
        row=2,
        col=1,
    ) 



# Adjust layout for subplots
fig.update_layout(
    font=dict(color="#ffffff", family="Helvetica"),
    paper_bgcolor="rgba(0,0,0,0)",  # Transparent background
    plot_bgcolor="#162024",  
    height=total_height,
    xaxis_rangeslider_visible=False,
    modebar_bgcolor="rgba(0,0,0,0)",
    legend_tracegroupgap = 100,
)

#Ajustar el tamaño de cada sub_plot
fig.update_layout(
    yaxis1=dict(title="Precio",showticklabels=True,),
    yaxis2=dict(title="Balance del Mercado (%)",showticklabels=True,),
)
fig.update_xaxes(showline=False, linewidth=2,linecolor='#000000', gridcolor='rgba(0,0,0,0)')
fig.update_yaxes(showline=False, linewidth=2, zeroline= False, linecolor='#ff0000', gridcolor='rgba(50,50,50,255)')     
chart = fig.show(config = {'scrollZoom': True, })